In [ ]:
import torch
print(torch.__version__)

2.5.1+cu121


In [ ]:
pip install torchinfo

In [ ]:
import torch
import torch.nn as nn

# Assuming Mamba is defined in the mamba_ssm module
from mamba_ssm import Mamba

# Define the wrapper class
class MambaWithClassification(nn.Module):
    def __init__(self, mamba_model, num_classes):
        super(MambaWithClassification, self).__init__()
        self.mamba_model = mamba_model
        self.classification_layer = nn.Linear(mamba_model.d_model, num_classes)

    def forward(self, x):
        x = self.mamba_model(x)
        # Apply classification layer on the last dimension
        x = self.classification_layer(x)
        return x

num_classes = 5  # Change this to the number of classes in your classification task
dim = 5
# Instantiate the Mamba model
mamba_model = Mamba(
    d_model=dim,  # Model dimension d_model
    d_state=16,   # SSM state expansion factor
    d_conv=4,     # Local convolution width
    expand=2      # Block expansion factor
).to("cuda")

# Wrap the Mamba model with the classification layer
model = MambaWithClassification(mamba_model, num_classes).to("cuda")

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to("cuda"), labels.to("cuda")

        optimizer.zero_grad()

        outputs = model(inputs)
        print("OT")
        print(outputs)
        print("Fine OT")
        # Assuming the output is of shape (batch_size, sequence_length, num_classes)
        outputs = outputs.mean(dim=1)  # Average over sequence length to match labels shape
        print("AfterMean")
        print(outputs)
        print("Fine AfterMean")
        print("label")
        print(labels)
        print("fine labels")
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("Finished Training")